In [1]:
# Necessary modules
import pandas as pd
import numpy  as np

from keras.models     import Sequential
from keras.layers     import Dense, Dropout, Activation, Flatten
from keras.layers     import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
# Get the data
datas = []
foldsToRanges = []
start, end = 0, 0
for fold in range(5):
    path = "../data/face_image_project/fold_%d_data.txt" % (fold)
    data_fold = pd.read_csv(filepath_or_buffer=path, sep="\t")
    data_fold = data_fold.sample(frac=1).reset_index(drop=True)    
    datas.append(data_fold)
    
    start = end
    end = start + len(data_fold)
    foldsToRanges.append(slice(start, end))

In [153]:
foldsToRanges

[slice(0, 4484, None),
 slice(4484, 8214, None),
 slice(8214, 12108, None),
 slice(12108, 15554, None),
 slice(15554, 19370, None)]

In [3]:
data = pd.concat(datas, ignore_index=True)
data.count()
(data.age == 'None').sum()

748

In [4]:
data = data[["user_id","original_image","face_id","gender", "age"]]
data.head()

,user_id,original_image,face_id,gender,age
0,10044155@N06,11343848026_7f024ce9a7_o.jpg,132,f,"(25, 32)"
1,29671106@N00,9058761563_8ba72a567a_o.jpg,188,m,"(38, 43)"
2,30601258@N03,10941877764_e2cd5c8f74_o.jpg,7,f,"(25, 32)"
3,113804525@N05,11847274825_66dcf9b574_o.jpg,322,f,55
4,113528649@N08,11778945623_801a991114_o.jpg,334,NaN,None


In [5]:
data = data[data.gender != "u"]
data.gender.value_counts()

f    9372
m    8120
Name: gender, dtype: int64

In [6]:
data.loc[data.gender == "f", "gender"] = 0
data.loc[data.gender == "m", "gender"] = 1

In [7]:
path_template = "../data/face_image_project/aligned/%s/landmark_aligned_face.%d.%s"
data["file_path"] = data[["user_id","face_id","original_image"]].apply(lambda x:  
                                                                   path_template % (x[0],x[1],x[2]),
                                                                   axis=1)

In [8]:
# Copy original before we drop stuff
data_orig = data

In [9]:
## Drop the None data
data = data[data.age != 'None']

In [10]:
# Building a vainilla CNN
vainilla_cnn = Sequential()
vainilla_cnn.add(Convolution2D(32, 3, 3, input_shape=(227,227,3)))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(Convolution2D(64, 3, 3))
vainilla_cnn.add(Activation('relu'))
vainilla_cnn.add(MaxPooling2D(pool_size=(2, 2)))
vainilla_cnn.add(Flatten())
vainilla_cnn.add(Dense(output_dim=200, input_dim=500))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=100, input_dim=200))
vainilla_cnn.add(BatchNormalization())
vainilla_cnn.add(Activation("relu"))
vainilla_cnn.add(Dense(output_dim=28))
vainilla_cnn.add(Activation("softmax"))

In [11]:
vainilla_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
data = data[["file_path","gender", "age"]]
data = pd.get_dummies(data, columns=["age"])

In [13]:
data.columns

Index([u'file_path', u'gender', u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(27, 32)', u'age_(38, 42)', u'age_(38, 43)',
       u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)', u'age_(60, 100)',
       u'age_(8, 12)', u'age_(8, 23)', u'age_13', u'age_2', u'age_22',
       u'age_23', u'age_29', u'age_3', u'age_32', u'age_34', u'age_35',
       u'age_36', u'age_42', u'age_45', u'age_46', u'age_55', u'age_56',
       u'age_57', u'age_58'],
      dtype='object')

In [14]:
age_cols = [u'age_(0, 2)', u'age_(15, 20)',
       u'age_(25, 32)', u'age_(27, 32)', u'age_(38, 42)', u'age_(38, 43)',
       u'age_(38, 48)', u'age_(4, 6)', u'age_(48, 53)', u'age_(60, 100)',
       u'age_(8, 12)', u'age_(8, 23)', u'age_13', u'age_2', u'age_22',
       u'age_23', u'age_29', u'age_3', u'age_34', u'age_35', u'age_36',
       u'age_42', u'age_45', u'age_46', u'age_55', u'age_56', u'age_57',
       u'age_58']

In [15]:
import scipy.misc

i_width  = 227
i_height = 227

def train_generator(data):
    while True:
        start, end = 0, 32
        while end < len(data):
            data = data.sample(frac=1).reset_index(drop=True)
            sample  = data[start:end]

            X = pd.DataFrame(sample["file_path"].apply(lambda x:  img_to_array( scipy.misc.imresize(load_img(x), (i_height, i_width) ) ) ))           
            X = X["file_path"].apply(lambda x: x.reshape((1,)+ x.shape))
            X = np.vstack(X)

            Y = sample[age_cols].as_matrix()
            
            yield (X, Y)
            start += 32
            end += 32

In [16]:
a,b = train_generator(data).next()
print a.shape
print b.shape

(32, 227, 227, 3)
(32, 28)


In [17]:
data_orig = data_orig[["file_path","gender", "age"]]
data_orig = pd.get_dummies(data_orig, columns=["age"])

fold0to3 = pd.concat([data_orig[foldsToRanges[0]], data_orig[foldsToRanges[1]], data_orig[foldsToRanges[2]], 
                     data_orig[foldsToRanges[3]]])
fold0to3 = fold0to3[fold0to3.age_None != 1]
fold0to3.head()

,file_path,gender,"age_(0, 2)","age_(15, 20)","age_(25, 32)","age_(27, 32)","age_(38, 42)","age_(38, 43)","age_(38, 48)","age_(4, 6)",...,age_35,age_36,age_42,age_45,age_46,age_55,age_56,age_57,age_58,age_None
0,../data/face_image_project/aligned/10044155@N0...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,../data/face_image_project/aligned/29671106@N0...,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,../data/face_image_project/aligned/30601258@N0...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,../data/face_image_project/aligned/113804525@N...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,../data/face_image_project/aligned/63153065@N0...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
history = vainilla_cnn.fit_generator(train_generator(fold0to3), samples_per_epoch=64, nb_epoch=2000)
vainilla_cnn.save_weights('vainilla_cnn_age_1_trainfold0to3.h5')

Epoch 1/2000
64/64 [==============================] - 6s - loss: 4.0058 - acc: 0.0312     
Epoch 2/2000
64/64 [==============================] - 1s - loss: 3.7889 - acc: 0.0312     
Epoch 3/2000
64/64 [==============================] - 1s - loss: 3.5257 - acc: 0.0938     
Epoch 4/2000
64/64 [==============================] - 1s - loss: 3.6495 - acc: 0.0625     
Epoch 5/2000
64/64 [==============================] - 1s - loss: 3.4443 - acc: 0.0312         
Epoch 6/2000
64/64 [==============================] - 1s - loss: 3.4960 - acc: 0.0469     

In [22]:
### UNCOMMENT ME NEXT TIME U RUN. DONT TRAIN THE DAMN NET AGAIN
#vainilla_cnn.load_weights('vainilla_cnn_age_1_trainfold0to3.h5')

### Test

In [19]:
fold4 = data_orig[foldsToRanges[4]]
fold4 = fold4[fold4.age_None != 1]
fold4.head()

,file_path,gender,"age_(0, 2)","age_(15, 20)","age_(25, 32)","age_(27, 32)","age_(38, 42)","age_(38, 43)","age_(38, 48)","age_(4, 6)",...,age_35,age_36,age_42,age_45,age_46,age_55,age_56,age_57,age_58,age_None
16440,../data/face_image_project/aligned/10069023@N0...,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16441,../data/face_image_project/aligned/115321157@N...,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16442,../data/face_image_project/aligned/10280355@N0...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16444,../data/face_image_project/aligned/10062073@N0...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
16445,../data/face_image_project/aligned/10280355@N0...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
len(fold4)

2664

In [21]:
vainilla_cnn.evaluate_generator(train_generator(fold4), 32*400)

[2.9907440868020059, 0.39695312500000002]